In [1]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Read the CSV file into a pandas dataframe
df = pd.read_csv("/content/dataset_file_name")

df["completion"] = df["completion"].replace({1: True, 0: False})

print(df)

In [ ]:
df.to_json("Dataset_file_name_jsonformat.jsonl", orient='records', lines=True)#converting dataframe to json file

In [ ]:
!openai tools fine_tunes.prepare_data -f Dataset_file_name_jsonformat.jsonl -q #preprocessing the data and to split the data into training and validation file

In [ ]:
#finetuning the ada model with train and validation data files with parameter set to 4 epoch, where we can change it to n epochs.
import openai
import os
os.environ["OPENAI_API_KEY"] = "OPEN_API_KEY_VALUE"
!openai api fine_tunes.create -t "new_prepared_train.jsonl" -v "new_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " True" -m ada  --n_epochs 4

In [ ]:
!openai api fine_tunes.follow -i 'fine-tune-ID' #you can view the fine-tuning process completion step by step.

In [ ]:
results = pd.read_csv('new_train_result_file.csv')#results of training can be obtained an stored in new file
results[results['classification/accuracy'].notnull()].tail(1)

In [ ]:
results[results['classification/accuracy'].notnull()]['classification/accuracy'].plot()# plotting the classification accuracy

In [ ]:
#function to pass the prompt as the parameter and returns completion as the output
def opai(m):
 openai.api_key = "API_KEY_VALUE"
 model_engine = 'MODEL_NAME'#enter the obtained model name in order to check the working of the model.
 prompt=m
 completions = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=1,
    logprobs=2,
    temperature=0
 )

 message = completions.choices[0].text

 return message

In [ ]:
#testing of model with test dataset
import os
import glob
import shutil
import pandas as pd
import shutil
import openai
from urllib import request
df = pd.read_csv('/content/test_data_file.csv')#insert test dataset file here
df.columns
df['model_4_epch']=' '
for index,rows in df.iterrows():
  m=df['prompt'].values[index]
  m= m+'->'
  res=opai(m)
  df['model_4_epch'].values[index]=res
df.to_csv('test_results.csv')#converting dataframe to csv file which is test results stored in the file 'test_results.csv'.

In [ ]:
#plotting the confusion matrix for particular model
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

df = pd.read_csv('/content/test_results.csv')#give test results file name

df["column_model_name"] = df["column_model_name"].apply(lambda x: True if x == " True" else False)#enter cloumn name of themodel result stored in test_results.csv file.
labels = df['completion'].values
predictions = df['column_model_name'].values

# Get confusion matrix with labels in correct order
cm = confusion_matrix(labels, predictions, labels=[True, False])
cm = cm[[0, 1], :]
cm = cm[:, [0, 1]]

# Plot confusion matrix as heatmap
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=["True", "False"], yticklabels=["True", "False"])
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()